# Example model ATP hydrolysis

Objective: Create a simple model of spontaneous ATP hydrolysis using mass action kinetics, independent of any isotope labeling.

## Model definition

In [ ]:
from isrene.model_definition import *
from isrene import ureg, Q_
from isrene.model_definition.model_export_sbml import ModelExporterSbmlMulti
from isrene.sbml_multi.sbml_multi_expand import NetworkGenerator
from pprint import pprint

In [ ]:
# Create a Model as container for all model components
model = Model("atp_hydrolysis")

# Create a 3D compartment
compartment = Compartment("C", ureg.Quantity(1, ureg.mL), 3, model=model)

# Create SpeciesTypes for all metabolites
ATP = SpeciesType("ATP", model=model)
ADP = SpeciesType("ADP", model=model)
P = SpeciesType("P", model=model)
H2O = SpeciesType("H2O", model=model)

# ignore H2O concentration in rate expressions
model.ignored_reactants = {H2O}

AtomMappingReaction(
    "hydrolysis",
    [ATP(compartment=compartment), H2O(compartment=compartment)],
    [ADP(compartment=compartment), P(compartment=compartment)],
    reversible=True,
    rate_law_generator=MassActionRateLawGenerator(),
)

InitialConcentration(ATP(compartment=compartment), Q_(1.0, ureg.mM))
InitialConcentration(ADP(compartment=compartment), Q_(1.0, ureg.mM))
InitialConcentration(P(compartment=compartment), Q_(1.0, ureg.mM))
InitialConcentration(
    H2O(compartment=compartment), Q_(1.0, ureg.mM), constant=True
);

## Write model to SBML multi and expand to SBML core

In [ ]:
# write SBML multi
multi_file = "tmp/atp_hydrolysis_multi_ma.xml"
exporter = ModelExporterSbmlMulti(model)
exporter.export(multi_file)

# expand network and save SBML core model
core_model_name = "tmp/atp_hydrolysis_core_ma.xml"
nwg = NetworkGenerator(model=multi_file)
nwg.create_sbml_core_model(core_model_name)

## Import to AMICI

In [ ]:
from amici.sbml_import import SbmlImporter

model_name = "atp_hydrolysis_model_ma"
output_dir = f"tmp/{model_name}"

SbmlImporter(core_model_name).sbml2amici(
    model_name=model_name, output_dir=output_dir, verbose=False
)

## Simulate model

In [ ]:
import amici

model_module = amici.import_model_module(model_name, module_path=output_dir)
amici_model = model_module.getModel()

pprint(amici_model.getParameterIds())

In [ ]:
# Set model parameters
amici_model.setParametersByIdRegex(".*", 1.0)
amici_model.setParameterById("ATP_C_C_initial_concentration", 0.9)
amici_model.setParameterById("ADP_C_C_initial_concentration", 0.2)
amici_model.setParameterById("P_C_C_initial_concentration", 1.1)
amici_model.setParameterById("H2O_C_C_initial_concentration", 1.0)
amici_model.setParameterById("kf_hydrolysis", 0.1)
amici_model.setParameterById("k_eq_hydrolysis", 1e5)

amici_model.setTimepoints(list(range(1000)))
solver = amici_model.getSolver()
rdata = amici.runAmiciSimulation(amici_model, solver)

from amici.plotting import plot_state_trajectories

plot_state_trajectories(rdata, model=amici_model)
# print(amici.getSimulationStatesAsDataFrame(amici_model, edata_list=[],
#                                           rdata_list=[rdata]))

idx_atp = list(amici_model.getStateIds()).index("ATP_C")
idx_adp = list(amici_model.getStateIds()).index("ADP_C")

assert rdata.x[-1, idx_atp] < rdata.x[-1, idx_adp]

In [ ]:
# Check that the steadystate matches the given equilibrium constant.

# The model uses concentrations in mM, but we assume equilibrium constants are computed from concentrations in M.
# mM -> M
conv = 1 / 1000
keq_apparent = (
    (conv * rdata.by_id("ADP_C") * conv * rdata.by_id("P_C"))
    / (conv * rdata.by_id("ATP_C"))
)[-1]
keq_expected = amici_model.getParameterById("k_eq_hydrolysis")
print(f"{keq_apparent=:.1f}\n{keq_expected=:.1f}")

assert (keq_expected - keq_apparent) / keq_expected < 0.01